In [1]:
import sys

from torch import nn
from torch.utils.data import DataLoader
from torchvision.models import mobilenet_v3_large
from torchvision.transforms import Resize
import torch

sys.path.insert(0, '..')

## View MobileNetV3 Architecture for Reference

In [2]:
# weights = MobileNet_V3_Large_Weights.IMAGENET1K_V2
model = mobilenet_v3_large(pretrained=True)

print(model)

MobileNetV3(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): ConvNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False

## Remove Last Layer

In [3]:
model_submodule_excl_last = list(model.children())[:-1]
model_submodule_excl_last = nn.Sequential(*model_submodule_excl_last)

model_submodule_excl_last

Sequential(
  (0): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): ConvNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      

In [4]:
model_submodule_excl_last(torch.rand(10, 3, 224, 224)).shape

torch.Size([10, 960, 1, 1])

In [5]:
model_last_submodule = list(list(model.children())[-1].children())[:-1]  # Remove last layer from last submodule
model_last_submodule = nn.Sequential(*model_last_submodule)

model_last_submodule

Sequential(
  (0): Linear(in_features=960, out_features=1280, bias=True)
  (1): Hardswish()
  (2): Dropout(p=0.2, inplace=True)
)

In [6]:
# output = model_submodule_excl_last(torch.rand(10, 3, 224, 224))
# model_last_submodule(output)

In [7]:
output = model_submodule_excl_last(torch.rand(10, 3, 224, 224))
output = torch.flatten(output, 1)
model_last_submodule(output).shape

torch.Size([10, 1280])

## Import `SiameseDuplicateImageNetwork` for Testing

In [8]:
from scripts.dataset_duplicate_image import DuplicateImageDataset
from scripts.model import SiameseDuplicateImageNetwork

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [10]:
model = SiameseDuplicateImageNetwork().to(device)

In [11]:
print(model)

SiameseDuplicateImageNetwork(
  (model): Sequential(
    (0): SiameseNetwork(
      (net1): Sequential(
        (0): Sequential(
          (0): Sequential(
            (0): ConvNormActivation(
              (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
              (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
              (2): Hardswish()
            )
            (1): InvertedResidual(
              (block): Sequential(
                (0): ConvNormActivation(
                  (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
                  (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
                  (2): ReLU(inplace=True)
                )
                (1): ConvNormActivation(
                  (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
                  (1): BatchNorm2d(16, eps=0.001, mom

In [12]:
train_dataset = DuplicateImageDataset('../data/Training Data', transforms=[Resize((960, 1280), antialias=True)])
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [13]:
feature1, feature2, _ = next(iter(train_dataloader))
feature1, feature2

(tensor([[[[119, 111, 105,  ..., 128, 128, 128],
           [117, 111, 107,  ..., 128, 128, 128],
           [121, 117, 112,  ..., 128, 128, 128],
           ...,
           [135, 132, 131,  ...,  63,  47,  34],
           [132, 133, 138,  ...,  63,  47,  34],
           [130, 135, 144,  ...,  63,  47,  34]],
 
          [[102,  94,  88,  ..., 128, 128, 129],
           [100,  94,  90,  ..., 128, 128, 129],
           [104, 100,  95,  ..., 128, 128, 128],
           ...,
           [122, 118, 118,  ...,  64,  48,  35],
           [119, 120, 125,  ...,  64,  48,  35],
           [117, 122, 131,  ...,  64,  48,  35]],
 
          [[ 94,  86,  80,  ..., 126, 126, 124],
           [ 92,  86,  82,  ..., 126, 126, 124],
           [ 94,  90,  85,  ..., 126, 126, 126],
           ...,
           [115, 110, 110,  ...,  68,  52,  39],
           [111, 112, 117,  ...,  68,  52,  39],
           [109, 114, 123,  ...,  68,  52,  39]]],
 
 
         [[[129, 129, 130,  ..., 135, 135, 134],
         

In [14]:
feature1 = feature1.to(device)
feature2 = feature2.to(device)

output = model(feature1, feature2)
print(output)

tensor([[0.0966],
        [0.0966],
        [0.0960],
        [0.0950],
        [0.0963],
        [0.0969],
        [0.0961],
        [0.0950],
        [0.0962],
        [0.0966],
        [0.0960],
        [0.0957],
        [0.0949],
        [0.0965],
        [0.0949],
        [0.0956],
        [0.0959],
        [0.0939],
        [0.0961],
        [0.0956],
        [0.0959],
        [0.0951],
        [0.0959],
        [0.0944],
        [0.0956],
        [0.0946],
        [0.0958],
        [0.0968],
        [0.0961],
        [0.0937],
        [0.0953],
        [0.0976],
        [0.0961],
        [0.0959],
        [0.0950],
        [0.0956],
        [0.0971],
        [0.0956],
        [0.0952],
        [0.0970],
        [0.0951],
        [0.0966],
        [0.0974],
        [0.0986],
        [0.0957],
        [0.0985],
        [0.0959],
        [0.0957],
        [0.0947],
        [0.0964],
        [0.0952],
        [0.0943],
        [0.0956],
        [0.0955],
        [0.0958],
        [0

In [15]:
output.shape

torch.Size([64, 1])